In [7]:
import openai

client = openai.Client()




In [9]:
assistant = client.beta.assistants.create(
    name="Tutor de Tecnologia",
    instructions="Você é um tutor sobre assuntos relacionados a tecnologia",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-0125"
)

In [10]:
pergunta ="""
Em  um sistema de chace com mapeaamento direto, dado uma memória principal de uma cache
de 256 Kb e blocos de 64 bytes
- Quantas linhas existem na cache ?
- Onde o bloco de endereço 0x1A23C será mapeado na cache ?
"""


In [12]:
# Criação da Thread

thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role ="user",
    content = pergunta
)

C:\Users\dyogo.almeida\AppData\Local\Temp\ipykernel_26596\143534826.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\dyogo.almeida\AppData\Local\Temp\ipykernel_26596\143534826.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [13]:
# Executa a Thread

run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions = "Nome do usuário premium "
)

C:\Users\dyogo.almeida\AppData\Local\Temp\ipykernel_26596\3010580843.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [14]:
run

Run(id='run_EofAfwHqCwUosH5IAci8vETG', assistant_id='asst_tsKiczhKim3uVqKLNafaP3nQ', cancelled_at=None, completed_at=None, created_at=1750789203, expires_at=1750789803, failed_at=None, incomplete_details=None, instructions='Nome do usuário premium ', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo-0125', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_tpL8EJfHGnhbCz38QL55dL9b', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={}, reasoning_effort=None)

In [17]:
import time

# Assuming 'client', 'thread', and 'run' are already defined from previous steps

while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id = run.id
    )

C:\Users\dyogo.almeida\AppData\Local\Temp\ipykernel_26596\860887447.py:7: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


In [18]:
if run.status =="completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")
                


C:\Users\dyogo.almeida\AppData\Local\Temp\ipykernel_26596\205122692.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  mensagens = client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_emKY2QN0ArufV2rtuTvV38JW', assistant_id='asst_tsKiczhKim3uVqKLNafaP3nQ', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Portanto:\n- O número de linhas na cache é de 4096.\n- O bloco de endereço 0x1A23C será mapeado na linha 572 da cache.'), type='text')], created_at=1750789211, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_EofAfwHqCwUosH5IAci8vETG', status=None, thread_id='thread_tpL8EJfHGnhbCz38QL55dL9b'), Message(id='msg_GYdjefsQxRSBLSupUX6EEh5r', assistant_id='asst_tsKiczhKim3uVqKLNafaP3nQ', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Para calcular a quantidade de linhas na cache em um sistema de mapeamento direto, podemos usar a fórmula:\n\n\\[ \\text{Número de Linhas} = \\frac{\\text{Tamanho da Cache}}{\\text{Tamanho do Bloco}} \\]\n\nDado que a memória principal da cache é

In [21]:
print(mensagens.data[0].content[0].text.value)

Portanto:
- O número de linhas na cache é de 4096.
- O bloco de endereço 0x1A23C será mapeado na linha 572 da cache.


# Analisando os Passos do Modelo


In [24]:
run_steps = client.beta.threads.runs.steps.list(
     thread_id=thread.id,
     run_id = run.id
)

C:\Users\dyogo.almeida\AppData\Local\Temp\ipykernel_26596\3582718740.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps = client.beta.threads.runs.steps.list(


In [27]:
for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("=" *10)
    if step.step_details.type == "message_creation": 
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id = step.step_details.message_creation.message_id
        )
        print(mensagens.data[0].content[0].text.value)


====Step message_creation


C:\Users\dyogo.almeida\AppData\Local\Temp\ipykernel_26596\712933490.py:9: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.retrieve(


Portanto:
- O número de linhas na cache é de 4096.
- O bloco de endereço 0x1A23C será mapeado na linha 572 da cache.

====Step tool_calls
# Dados do problema
tamanho_cache_kb = 256
tamanho_bloco_bytes = 64
endereco_bloco = 0x1A23C

# Convertendo o tamanho da cache para bytes
tamanho_cache_bytes = tamanho_cache_kb * 1024

# Calculando o número de linhas na cache
num_linhas_cache = tamanho_cache_bytes / tamanho_bloco_bytes

# Calculando em qual linha da cache o bloco de endereço será mapeado
linha_cache = endereco_bloco % num_linhas_cache

num_linhas_cache, linha_cache

====Step message_creation
Portanto:
- O número de linhas na cache é de 4096.
- O bloco de endereço 0x1A23C será mapeado na linha 572 da cache.
